# Building a CNN for object detection 

__Prerequisites__

- [Custom Datasets](https://github.com/AI-Core/Convolutional-Neural-Networks/blob/master/Custom%20Datasets.ipynb)

Lets build a CNN to fit the S40 Dataset which we built a dataset class for previously.

Our input is an image of different sizer varying around 400x400. The label for each image is 4 floats which represent the x and y value of the centre of the rectangle and as its height and width. The numbers are floats instead of integers as we do not directly use the pixel location as the label but normalize it w.r.t the height and width of the image. So our values range from 0-1.

In [1]:
# IMPORTS
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import S40dataset
import matplotlib.pyplot as plt
import time

In [8]:
#HYPERPARAMETER
batch_size = 4

transform = transforms.Compose([# compose transforms 
    transforms.Resize((64, 64)),# resize imgs
    transforms.ToTensor()# convert PIL images into Torch Tensors
])

# MAKE DATASET
dataset = S40dataset.S40dataset(transform=transform)# make the dataset and pass it our transforms

# SPLIT DATASET
train_len = int( 0.8 * len(dataset))
val_len = int( 0.1 * len(dataset))
test_len = len(dataset) - train_len - val_len
train_data, val_data, test_data = random_split(dataset, [train_len, val_len, test_len])# split the dataset into train, test, validation

# MAKE DATALOADERS
train_loader = DataLoader(train_data,# make the training dataloader
                          batch_size = batch_size,
                          shuffle=True)
test_loader = DataLoader(test_data,
                       batch_size=batch_size)

# SHOW A RANDOM EXAMPLE FROM THE DATASET
for i in test_loader:
    b = i
    break
import numpy as np
idx = np.random.randint(len(test_loader))
S40dataset.show(b)

In [17]:
# SET NUMBER OF FILTERS TO USE IN EACH CONV LAYER
channels1 = 128
channels2 = 64
channels3 = 64
channels4 = 64

# CREATE MODEL
class DetectCNN(torch.nn.Module):# create class
    def __init__(self):# initialise
        super().__init__()         # initialise parent class
        self.conv_layers = torch.nn.Sequential(        # make sequential model for conv layers
            torch.nn.Conv2d(3, channels1, 7),            # make layers
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(channels1),
            torch.nn.Dropout(0.2),
            torch.nn.Conv2d(channels1, channels2, 7),            # make layers
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(channels2),
            torch.nn.Dropout(0.2),
            torch.nn.Conv2d(channels2, channels3, 7),            # make layers
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(channels3),
            torch.nn.Dropout(0.2),
        )
        self.fc_layers = torch.nn.Sequential(# make sequential model for fully connected (linear) layers
            torch.nn.Linear(64*46*46, 4),# make layers
            torch.nn.Sigmoid()
        )
    
    def forward(self, x):# define forward pass
        x = self.conv_layers(x)# forward pass through conv layers
        #print(x.shape)
        x = x.view(-1, 64*46*46)# flatten conv output to pass to fully connected layers
        x = self.fc_layers(x)# pass through fc layers
        return x# return output

In [18]:
# HYPERPARAMETERS
epochs = 1
lr = 0.00001
        
cnn = DetectCNN()# create instance of model
criterion = torch.nn.MSELoss()# define loss function
optimiser = torch.optim.Adam(cnn.parameters(), lr=lr, weight_decay=1)# define optimiser

# SET UP TRAINING VISUALISATION
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # we will use this to show our models performance on a graph

# TRAIN
def train(model, epochs):
    for epoch in range(epochs):
        for idx, batch in enumerate(train_loader):
            x, bndbox = batch    # unpack batch
            pred_bndbox = model(x)# forward pass
            #print('label:', bndbox, 'prediction:', pred_bndbox)
            loss = criterion(pred_bndbox, bndbox)     # compute loss for this batch
            optimiser.zero_grad()# zero gradients of optimiser
            loss.backward()     # backward pass (find rate of change of loss with respect to model parameters)
            optimiser.step()# take optimisation step
            print('Epoch:', epoch, 'Batch:', idx, 'Loss:', loss.item())
            writer.add_scalar('DETECTION Loss/Train', loss, epoch*len(train_loader) + idx)    # write loss to a graph
    print('Training Complete. Final loss =',loss.item())

train(cnn, epochs)

Epoch: 0 Batch: 0 Loss: 0.05645712837576866
Epoch: 0 Batch: 1 Loss: 0.058021146804094315
Epoch: 0 Batch: 2 Loss: 0.04598623886704445
Epoch: 0 Batch: 3 Loss: 0.05713953450322151
Epoch: 0 Batch: 4 Loss: 0.06860405206680298
Epoch: 0 Batch: 5 Loss: 0.05349897965788841
Epoch: 0 Batch: 6 Loss: 0.029903464019298553
Epoch: 0 Batch: 7 Loss: 0.0407395139336586
Epoch: 0 Batch: 8 Loss: 0.07721612602472305
Epoch: 0 Batch: 9 Loss: 0.04646402224898338
Epoch: 0 Batch: 10 Loss: 0.07514435797929764
Epoch: 0 Batch: 11 Loss: 0.07771164178848267
Epoch: 0 Batch: 12 Loss: 0.07752968370914459
Epoch: 0 Batch: 13 Loss: 0.07032085210084915
Epoch: 0 Batch: 14 Loss: 0.1043890193104744
Epoch: 0 Batch: 15 Loss: 0.060622040182352066
Epoch: 0 Batch: 16 Loss: 0.03198594972491264
Epoch: 0 Batch: 17 Loss: 0.08520392328500748
Epoch: 0 Batch: 18 Loss: 0.04685831815004349
Epoch: 0 Batch: 19 Loss: 0.06573498249053955
Epoch: 0 Batch: 20 Loss: 0.07918959856033325
Epoch: 0 Batch: 21 Loss: 0.038920413702726364
Epoch: 0 Batch: 22

Epoch: 0 Batch: 181 Loss: 0.05752018839120865
Epoch: 0 Batch: 182 Loss: 0.060475751757621765
Epoch: 0 Batch: 183 Loss: 0.038449373096227646
Epoch: 0 Batch: 184 Loss: 0.04966718330979347
Epoch: 0 Batch: 185 Loss: 0.056388579308986664
Epoch: 0 Batch: 186 Loss: 0.09834236651659012
Epoch: 0 Batch: 187 Loss: 0.05006694048643112
Epoch: 0 Batch: 188 Loss: 0.058133941143751144
Epoch: 0 Batch: 189 Loss: 0.07794458419084549
Epoch: 0 Batch: 190 Loss: 0.03864437714219093
Epoch: 0 Batch: 191 Loss: 0.029693622142076492
Epoch: 0 Batch: 192 Loss: 0.049073025584220886
Epoch: 0 Batch: 193 Loss: 0.026825811713933945
Epoch: 0 Batch: 194 Loss: 0.021207889541983604
Epoch: 0 Batch: 195 Loss: 0.03589247167110443
Epoch: 0 Batch: 196 Loss: 0.07113071531057358
Epoch: 0 Batch: 197 Loss: 0.08218040317296982
Epoch: 0 Batch: 198 Loss: 0.056061528623104095
Epoch: 0 Batch: 199 Loss: 0.05423705279827118
Epoch: 0 Batch: 200 Loss: 0.0625477135181427
Epoch: 0 Batch: 201 Loss: 0.08892405778169632
Epoch: 0 Batch: 202 Loss: 

Epoch: 0 Batch: 359 Loss: 0.022718684747815132
Epoch: 0 Batch: 360 Loss: 0.028012510389089584
Epoch: 0 Batch: 361 Loss: 0.04265731945633888
Epoch: 0 Batch: 362 Loss: 0.02648826874792576
Epoch: 0 Batch: 363 Loss: 0.046955373138189316
Epoch: 0 Batch: 364 Loss: 0.045025356113910675
Epoch: 0 Batch: 365 Loss: 0.07774243503808975
Epoch: 0 Batch: 366 Loss: 0.03252518177032471
Epoch: 0 Batch: 367 Loss: 0.06517846882343292
Epoch: 0 Batch: 368 Loss: 0.0194143895059824
Epoch: 0 Batch: 369 Loss: 0.07880432903766632
Epoch: 0 Batch: 370 Loss: 0.02522585727274418
Epoch: 0 Batch: 371 Loss: 0.033788811415433884
Epoch: 0 Batch: 372 Loss: 0.029522260650992393
Epoch: 0 Batch: 373 Loss: 0.030848659574985504
Epoch: 0 Batch: 374 Loss: 0.04191112890839577
Epoch: 0 Batch: 375 Loss: 0.02486252225935459
Epoch: 0 Batch: 376 Loss: 0.05640795826911926
Epoch: 0 Batch: 377 Loss: 0.029687246307730675
Epoch: 0 Batch: 378 Loss: 0.050919778645038605
Epoch: 0 Batch: 379 Loss: 0.037190645933151245
Epoch: 0 Batch: 380 Loss:

Epoch: 0 Batch: 537 Loss: 0.01742064580321312
Epoch: 0 Batch: 538 Loss: 0.057741038501262665
Epoch: 0 Batch: 539 Loss: 0.06928199529647827
Epoch: 0 Batch: 540 Loss: 0.0449090301990509
Epoch: 0 Batch: 541 Loss: 0.04350493848323822
Epoch: 0 Batch: 542 Loss: 0.03579534962773323
Epoch: 0 Batch: 543 Loss: 0.04330960288643837
Epoch: 0 Batch: 544 Loss: 0.0262145958840847
Epoch: 0 Batch: 545 Loss: 0.06641042232513428
Epoch: 0 Batch: 546 Loss: 0.05726765841245651
Epoch: 0 Batch: 547 Loss: 0.03030535578727722
Epoch: 0 Batch: 548 Loss: 0.04407595843076706
Epoch: 0 Batch: 549 Loss: 0.02298636920750141
Epoch: 0 Batch: 550 Loss: 0.04390871152281761
Epoch: 0 Batch: 551 Loss: 0.04998801648616791
Epoch: 0 Batch: 552 Loss: 0.03379856050014496
Epoch: 0 Batch: 553 Loss: 0.06248883157968521
Epoch: 0 Batch: 554 Loss: 0.0504213348031044
Epoch: 0 Batch: 555 Loss: 0.03629360347986221
Epoch: 0 Batch: 556 Loss: 0.045268360525369644
Epoch: 0 Batch: 557 Loss: 0.037606727331876755
Epoch: 0 Batch: 558 Loss: 0.030528

Epoch: 0 Batch: 714 Loss: 0.04465160146355629
Epoch: 0 Batch: 715 Loss: 0.032749198377132416
Epoch: 0 Batch: 716 Loss: 0.012151009403169155
Epoch: 0 Batch: 717 Loss: 0.03802435100078583
Epoch: 0 Batch: 718 Loss: 0.07543972879648209
Epoch: 0 Batch: 719 Loss: 0.049289073795080185
Epoch: 0 Batch: 720 Loss: 0.03795953840017319
Epoch: 0 Batch: 721 Loss: 0.03157738968729973
Epoch: 0 Batch: 722 Loss: 0.04402218386530876
Epoch: 0 Batch: 723 Loss: 0.04654109477996826
Epoch: 0 Batch: 724 Loss: 0.03246879577636719
Epoch: 0 Batch: 725 Loss: 0.04031476378440857
Epoch: 0 Batch: 726 Loss: 0.02237522602081299
Epoch: 0 Batch: 727 Loss: 0.0430956669151783
Epoch: 0 Batch: 728 Loss: 0.0992661565542221
Epoch: 0 Batch: 729 Loss: 0.04860510677099228
Epoch: 0 Batch: 730 Loss: 0.02314595878124237
Epoch: 0 Batch: 731 Loss: 0.01325919758528471
Epoch: 0 Batch: 732 Loss: 0.026459667831659317
Epoch: 0 Batch: 733 Loss: 0.05900048464536667
Epoch: 0 Batch: 734 Loss: 0.06592798233032227
Epoch: 0 Batch: 735 Loss: 0.0478

...

Epoch: 0 Batch: 1067 Loss: 0.02932056598365307
Epoch: 0 Batch: 1068 Loss: 0.0338483527302742
Epoch: 0 Batch: 1069 Loss: 0.019878104329109192
Epoch: 0 Batch: 1070 Loss: 0.048047393560409546
Epoch: 0 Batch: 1071 Loss: 0.031149478629231453
Epoch: 0 Batch: 1072 Loss: 0.06300907582044601
Epoch: 0 Batch: 1073 Loss: 0.04093426465988159
Epoch: 0 Batch: 1074 Loss: 0.031193232163786888
Epoch: 0 Batch: 1075 Loss: 0.025483369827270508
Epoch: 0 Batch: 1076 Loss: 0.03660259023308754
Epoch: 0 Batch: 1077 Loss: 0.024725385010242462
Epoch: 0 Batch: 1078 Loss: 0.03519272804260254
Epoch: 0 Batch: 1079 Loss: 0.02898186817765236
Epoch: 0 Batch: 1080 Loss: 0.026245664805173874
Epoch: 0 Batch: 1081 Loss: 0.04582689329981804
Epoch: 0 Batch: 1082 Loss: 0.022914309054613113
Epoch: 0 Batch: 1083 Loss: 0.05234179273247719
Epoch: 0 Batch: 1084 Loss: 0.044978003948926926
Epoch: 0 Batch: 1085 Loss: 0.03216637298464775
Epoch: 0 Batch: 1086 Loss: 0.04336733743548393
Epoch: 0 Batch: 1087 Loss: 0.08117985725402832
Epoch

Epoch: 0 Batch: 1240 Loss: 0.016006719321012497
Epoch: 0 Batch: 1241 Loss: 0.015194284729659557
Epoch: 0 Batch: 1242 Loss: 0.05212243273854256
Epoch: 0 Batch: 1243 Loss: 0.028561251237988472
Epoch: 0 Batch: 1244 Loss: 0.04847898706793785
Epoch: 0 Batch: 1245 Loss: 0.059548549354076385
Epoch: 0 Batch: 1246 Loss: 0.02658136934041977
Epoch: 0 Batch: 1247 Loss: 0.06724531203508377
Epoch: 0 Batch: 1248 Loss: 0.04363342747092247
Epoch: 0 Batch: 1249 Loss: 0.04876433312892914
Epoch: 0 Batch: 1250 Loss: 0.020390236750245094
Epoch: 0 Batch: 1251 Loss: 0.018612533807754517
Epoch: 0 Batch: 1252 Loss: 0.020530052483081818
Epoch: 0 Batch: 1253 Loss: 0.04133473336696625
Epoch: 0 Batch: 1254 Loss: 0.057245828211307526
Epoch: 0 Batch: 1255 Loss: 0.025380849838256836
Epoch: 0 Batch: 1256 Loss: 0.029504448175430298
Epoch: 0 Batch: 1257 Loss: 0.03269810229539871
Epoch: 0 Batch: 1258 Loss: 0.03463446721434593
Epoch: 0 Batch: 1259 Loss: 0.02190873585641384
Epoch: 0 Batch: 1260 Loss: 0.03849859908223152
Epo

Epoch: 0 Batch: 1413 Loss: 0.046319782733917236
Epoch: 0 Batch: 1414 Loss: 0.03167933598160744
Epoch: 0 Batch: 1415 Loss: 0.058712683618068695
Epoch: 0 Batch: 1416 Loss: 0.038792259991168976
Epoch: 0 Batch: 1417 Loss: 0.01847345381975174
Epoch: 0 Batch: 1418 Loss: 0.04764668643474579
Epoch: 0 Batch: 1419 Loss: 0.02794520929455757
Epoch: 0 Batch: 1420 Loss: 0.045102525502443314
Epoch: 0 Batch: 1421 Loss: 0.016820495948195457
Epoch: 0 Batch: 1422 Loss: 0.03064187988638878
Epoch: 0 Batch: 1423 Loss: 0.027175676077604294
Epoch: 0 Batch: 1424 Loss: 0.030716361477971077
Epoch: 0 Batch: 1425 Loss: 0.008811072446405888
Epoch: 0 Batch: 1426 Loss: 0.06248731538653374
Epoch: 0 Batch: 1427 Loss: 0.026955924928188324
Epoch: 0 Batch: 1428 Loss: 0.01834421046078205
Epoch: 0 Batch: 1429 Loss: 0.027731142938137054
Epoch: 0 Batch: 1430 Loss: 0.043477270752191544
Epoch: 0 Batch: 1431 Loss: 0.040627796202898026
Epoch: 0 Batch: 1432 Loss: 0.03709237277507782
Epoch: 0 Batch: 1433 Loss: 0.03945913910865784
E

Epoch: 0 Batch: 1587 Loss: 0.023649340495467186
Epoch: 0 Batch: 1588 Loss: 0.06275204569101334
Epoch: 0 Batch: 1589 Loss: 0.03149007260799408
Epoch: 0 Batch: 1590 Loss: 0.04631602764129639
Epoch: 0 Batch: 1591 Loss: 0.049291521310806274
Epoch: 0 Batch: 1592 Loss: 0.03203799948096275
Epoch: 0 Batch: 1593 Loss: 0.06294029206037521
Epoch: 0 Batch: 1594 Loss: 0.026339901611208916
Epoch: 0 Batch: 1595 Loss: 0.03409384936094284
Epoch: 0 Batch: 1596 Loss: 0.01626732386648655
Epoch: 0 Batch: 1597 Loss: 0.025852391496300697
Epoch: 0 Batch: 1598 Loss: 0.04989103227853775
Epoch: 0 Batch: 1599 Loss: 0.03882082551717758
Epoch: 0 Batch: 1600 Loss: 0.0233151838183403
Epoch: 0 Batch: 1601 Loss: 0.03342292457818985
Epoch: 0 Batch: 1602 Loss: 0.04736480861902237
Epoch: 0 Batch: 1603 Loss: 0.0538935512304306
Epoch: 0 Batch: 1604 Loss: 0.047707103192806244
Epoch: 0 Batch: 1605 Loss: 0.04579741135239601
Epoch: 0 Batch: 1606 Loss: 0.021070504561066628
Epoch: 0 Batch: 1607 Loss: 0.037495654076337814
Epoch: 0

Epoch: 0 Batch: 1761 Loss: 0.04058777168393135
Epoch: 0 Batch: 1762 Loss: 0.020127370953559875
Epoch: 0 Batch: 1763 Loss: 0.021601233631372452
Epoch: 0 Batch: 1764 Loss: 0.028828972950577736
Epoch: 0 Batch: 1765 Loss: 0.05255467817187309
Epoch: 0 Batch: 1766 Loss: 0.0400860421359539
Epoch: 0 Batch: 1767 Loss: 0.016675131395459175
Epoch: 0 Batch: 1768 Loss: 0.021932097151875496
Epoch: 0 Batch: 1769 Loss: 0.06403866410255432
Epoch: 0 Batch: 1770 Loss: 0.011339913122355938
Epoch: 0 Batch: 1771 Loss: 0.04495532438158989
Epoch: 0 Batch: 1772 Loss: 0.015585104934871197
Epoch: 0 Batch: 1773 Loss: 0.020594298839569092
Epoch: 0 Batch: 1774 Loss: 0.05312282592058182
Epoch: 0 Batch: 1775 Loss: 0.04107295349240303
Epoch: 0 Batch: 1776 Loss: 0.01594441384077072
Epoch: 0 Batch: 1777 Loss: 0.04320300370454788
Epoch: 0 Batch: 1778 Loss: 0.03814433515071869
Epoch: 0 Batch: 1779 Loss: 0.028325513005256653
Epoch: 0 Batch: 1780 Loss: 0.022643353790044785
Epoch: 0 Batch: 1781 Loss: 0.02509240247309208
Epoc

In [ ]:
torch.save(cnn.state_dict(), str(time.time()))# save model

In [ ]:
def visualise(model, n=10):
    model.eval()
    for idx, batch in enumerate(test_loader):
        print(type(batch))
        x, y = batch
        pred_bndbox = model(x)
        S40dataset.show(batch, pred_bndbox=pred_bndbox)
        if idx == n:
            break

In [ ]:
visualise(cnn, n=10)

## Next steps
- [Recurrent Neural Networks](https://github.com/AI-Core/Recurrent-Neural-Networks/blob/master/Recurrent%20Neural%20Networks.ipynb)